# sqlok diag (`Microsoft SQL Server 2022`)

> `query store`

> `query execution plan`

> `cached plans`

* Створення БД
* Наповнення БД
* Розбір планів, пошук зауважень та пропущених індексів

## Створення БД

In [2]:
CREATE DATABASE [sqlok_diag];
GO

USE sqlok_diag;
GO

CREATE TABLE dbo.cached_plans (
	bucketid INT NOT NULL,
	refcounts INT NOT NULL,
	usecounts INT NOT NULL,
	size_in_bytes INT NOT NULL,
	cacheobjtype NVARCHAR(50) NOT NULL,
	objtype NVARCHAR(20) NOT NULL,
	query_plan XML NULL,
	objectid INT NULL,
	dbid SMALLINT NULL
) WITH (DATA_COMPRESSION=PAGE);

CREATE TABLE dbo.query_plan_propterties (
	database_id SMALLINT NOT NULL,
	plan_id BIGINT NOT NULL,
	query_id BIGINT NOT NULL,
	plan_group_id BIGINT NULL,
	engine_version NVARCHAR(32) NULL,
	compatibility_level SMALLINT NOT NULL,
	query_plan_hash BINARY(8) NOT NULL,
	is_online_index_plan BIT NOT NULL,
	is_trivial_plan BIT NOT NULL,
	is_parallel_plan BIT NOT NULL,
	is_forced_plan BIT NOT NULL,
	is_natively_compiled BIT NOT NULL,
	force_failure_count BIGINT NOT NULL,
	last_force_failure_reason INT NOT NULL,
	last_force_failure_reason_desc NVARCHAR(128) NULL,
	count_compiles BIGINT NULL,
	initial_compile_start_time DATETIMEOFFSET(7) NOT NULL,
	last_compile_start_time DATETIMEOFFSET(7) NULL,
	last_execution_time DATETIMEOFFSET(7) NULL,
	avg_compile_duration FLOAT NULL,
	last_compile_duration BIGINT NULL,
	plan_forcing_type INT NOT NULL,
	plan_forcing_type_desc NVARCHAR(60) NULL,
	has_compile_replay_script BIT NOT NULL,
	is_optimized_plan_forcing_disabled BIT NOT NULL,
	plan_type INT NOT NULL,
	plan_type_desc NVARCHAR(60) NULL
);

CREATE CLUSTERED INDEX ci ON query_plan_propterties(database_id, plan_id, query_id, plan_group_id)WITH(DATA_COMPRESSION = PAGE);


CREATE TABLE dbo.query_plan_relop (
	database_id SMALLINT NOT NULL,
    relop_id BIGINT IDENTITY(1, 1) NOT NULL,
	parent_relop_id BIGINT NULL,
	plan_id BIGINT NULL,
	StatementId INT NOT NULL,
	StatementCompId INT NOT NULL,
	QueryHash VARCHAR(16) NULL,
	QueryPlanHash VARCHAR(16) NULL,
	lvl INT NOT NULL,
	ParentNodeId INT NULL,
	NodeId INT NULL,
	PhysicalOp NVARCHAR(256) NULL,
	LogicalOp NVARCHAR(256) NULL,
	EstimateRows FLOAT NULL,
	EstimatedRowsRead FLOAT NULL,
	EstimateIO FLOAT NULL,
	EstimateCPU FLOAT NULL,
	AvgRowSize FLOAT NULL,
	EstimatedTotalSubtreeCost FLOAT NULL,
	TableCardinality FLOAT NULL,
	Parallel INT NULL,
	EstimateRebinds FLOAT NULL,
	EstimateRewinds FLOAT NULL,
	EstimatedExecutionMode NVARCHAR(128) NULL,
	db NVARCHAR(128) NULL,
	sch NVARCHAR(128) NULL,
	tab NVARCHAR(128) NULL,
	idx NVARCHAR(128) NULL,
	alias NVARCHAR(128) NULL,
	IndexKind NVARCHAR(128) NULL,
	Storage NVARCHAR(128) NULL
);

CREATE CLUSTERED INDEX ci ON dbo.query_plan_relop(plan_id ASC, StatementId ASC, StatementCompId ASC, ParentNodeId ASC, NodeId ASC) WITH (DATA_COMPRESSION=PAGE);


CREATE TABLE dbo.query_plan_statement (
	database_id SMALLINT NOT NULL,
    plan_id BIGINT NULL,
	StatementId INT NOT NULL,
	StatementCompId INT NOT NULL,
	StatementType NVARCHAR(128) NULL,
	StatementSqlHandle VARCHAR(128) NULL,
	DatabaseContextSettingsId INT NULL,
	ParentObjectId INT NULL,
	StatementParameterizationType INT NULL,
	RetrievedFromCache BIT NULL,
	StatementSubTreeCost FLOAT NULL,
	StatementEstRows FLOAT NULL,
	StatementOptmLevel VARCHAR(16) NULL,
	QueryHash VARCHAR(16) NULL,
	QueryPlanHash VARCHAR(16) NULL,
	CardinalityEstimationModelVersion INT NULL,
	StatementText NVARCHAR(MAX) NULL,
	CachedPlanSize BIGINT NULL,
	CompileTime BIGINT NULL,
	CompileCPU BIGINT NULL,
	CompileMemory BIGINT NULL,
	SerialRequiredMemory BIGINT NULL,
	SerialDesiredMemory BIGINT NULL,
	GrantedMemory BIGINT NULL,
	MaxUsedMemory BIGINT NULL,
	EstimatedAvailableMemoryGrant BIGINT NULL,
	EstimatedPagesCached BIGINT NULL,
	EstimatedAvailableDegreeOfParallelism BIGINT NULL,
	MaxCompileMemory BIGINT NULL
);

CREATE CLUSTERED INDEX ci ON dbo.query_plan_statement(database_id, plan_id ASC) WITH (DATA_COMPRESSION=PAGE);


CREATE TABLE dbo.query_store_query_plan (database_id SMALLINT NOT NULL, plan_id BIGINT NOT NULL, query_plan XML NULL, PRIMARY KEY (database_id, plan_id)) WITH (DATA_COMPRESSION=PAGE);

CREATE PRIMARY XML INDEX pxi ON dbo.query_store_query_plan(query_plan);



TypeError: Cannot read properties of undefined (reading 'error')

## Наповнення БД
### Кеш планів (`cached_plans`)

In [2]:
use sqlok_diag;
GO

INSERT	cached_plans(bucketid, refcounts, usecounts, size_in_bytes, cacheobjtype, objtype, query_plan, objectid, dbid)
SELECT
	cp.bucketid,
	cp.refcounts,
	cp.usecounts,
	cp.size_in_bytes,
	cp.cacheobjtype,
	cp.objtype,
	qp.query_plan,
	qp.objectid,
	qp.dbid
FROM	sys.dm_exec_cached_plans cp
	CROSS APPLY sys.dm_exec_query_plan(cp.plan_handle) qp
WHERE qp.query_plan IS NOT NULL;



### Плани із Query store (`query_store_query_plan`)
При копіюванні із Query Store для кожної БД буде виконано наступний запит:
```sql
INSERT query_store_query_plan (database_id, plan_id, query_plan)
SELECT DB_ID('MonkLab') database_id, qsp.plan_id, CONVERT(XML, qsp.query_plan) query_plan
FROM [MonkLab].sys.query_store_plan qsp
WHERE NOT EXISTS (SELECT * FROM query_store_query_plan WHERE database_id = DB_ID('MonkLab') AND plan_id = qsp.plan_id);
```

In [ ]:
DECLARE @copy_query_store_plans NVARCHAR(MAX) = N'';
SELECT	@copy_query_store_plans += REPLACE('
INSERT query_store_query_plan (database_id, plan_id, query_plan)
SELECT DB_ID(''@name'') database_id, qsp.plan_id, CONVERT(XML, qsp.query_plan) query_plan
FROM [@name].sys.query_store_plan qsp
WHERE NOT EXISTS (SELECT * FROM	query_store_query_plan WHERE database_id = DB_ID(''@name'') AND plan_id = qsp.plan_id);
', '@name', name)
FROM	sys.databases
WHERE state = 0 AND is_read_only = 0 AND user_access = 0 AND is_query_store_on = 1 AND database_id > 4;
PRINT @copy_query_store_plans;

## Розбір планів, пошук зауважень та пропущених індексів

### Query Store

#### query_plan_statement

In [ ]:
WITH XMLNAMESPACES('http://schemas.microsoft.com/sqlserver/2004/07/showplan' AS xs)
INSERT	query_plan_statement(database_id, plan_id, StatementId, StatementCompId, StatementType, StatementSqlHandle, DatabaseContextSettingsId, ParentObjectId, StatementParameterizationType, RetrievedFromCache, StatementSubTreeCost, StatementEstRows, StatementOptmLevel, QueryHash, QueryPlanHash, CardinalityEstimationModelVersion, StatementText, CachedPlanSize, CompileTime, CompileCPU, CompileMemory, SerialRequiredMemory, SerialDesiredMemory, GrantedMemory, MaxUsedMemory, EstimatedAvailableMemoryGrant, EstimatedPagesCached, EstimatedAvailableDegreeOfParallelism, MaxCompileMemory)
SELECT
	c.database_id,
	c.plan_id,
	s.value('@StatementId', 'INT') StatementId,
	s.value('@StatementCompId', 'INT') StatementCompId,
	s.value('@StatementType', 'nvarchar(128)') StatementType,
	s.value('@StatementSqlHandle', 'varchar(128)') StatementSqlHandle,
	s.value('@DatabaseContextSettingsId', 'INT') DatabaseContextSettingsId,
	s.value('@ParentObjectId', 'INT') ParentObjectId,
	s.value('@StatementParameterizationType', 'INT') StatementParameterizationType,
	s.value('@RetrievedFromCache', 'BIT') RetrievedFromCache,
	s.value('@StatementSubTreeCost', 'float') StatementSubTreeCost,
	s.value('@StatementEstRows', 'float') StatementEstRows,
	s.value('@StatementOptmLevel', 'varchar(16)') StatementOptmLevel,
	s.value('@QueryHash', 'varchar(16)') QueryHash,
	s.value('@QueryPlanHash', 'varchar(16)') QueryPlanHash,
	s.value('@CardinalityEstimationModelVersion', 'INT') CardinalityEstimationModelVersion,
	s.value('@StatementText', 'nvarchar(max)') StatementText,
	p.value('@CachedPlanSize', 'BIGINT') CachedPlanSize,
	p.value('@CompileTime', 'BIGINT') CompileTime,
	p.value('@CompileCPU', 'BIGINT') CompileCPU,
	p.value('@CompileMemory', 'BIGINT') CompileMemory,
	p.value('xs:MemoryGrantInfo[1]/@SerialRequiredMemory', 'BIGINT') SerialRequiredMemory,
	p.value('xs:MemoryGrantInfo[1]/@SerialDesiredMemory', 'BIGINT') SerialDesiredMemory,
	p.value('xs:MemoryGrantInfo[1]/@GrantedMemory', 'BIGINT') GrantedMemory,
	p.value('xs:MemoryGrantInfo[1]/@MaxUsedMemory', 'BIGINT') MaxUsedMemory,
	p.value('xs:OptimizerHardwareDependentProperties[1]/@EstimatedAvailableMemoryGrant', 'BIGINT') EstimatedAvailableMemoryGrant,
	p.value('xs:OptimizerHardwareDependentProperties[1]/@EstimatedPagesCached', 'BIGINT') EstimatedPagesCached,
	p.value('xs:OptimizerHardwareDependentProperties[1]/@EstimatedAvailableDegreeOfParallelism', 'BIGINT') EstimatedAvailableDegreeOfParallelism,
	p.value('xs:OptimizerHardwareDependentProperties[1]/@MaxCompileMemory', 'BIGINT') MaxCompileMemory
FROM	query_store_query_plan c
	CROSS APPLY c.query_plan.nodes('/xs:ShowPlanXML/xs:BatchSequence/xs:Batch/xs:Statements/*') n(s)
	OUTER APPLY s.nodes('xs:QueryPlan') q(p);


#### query_plan_relop

In [ ]:
WITH XMLNAMESPACES('http://schemas.microsoft.com/sqlserver/2004/07/showplan' AS xs)
, relop AS (
	SELECT
		c.database_id,
		c.plan_id,
		x.value('@StatementId', 'INT') StatementId,
		x.value('@StatementCompId', 'INT') StatementCompId,
		x.value('@QueryHash', 'varchar(16)') QueryHash,
		x.value('@QueryPlanHash', 'varchar(16)') QueryPlanHash,
		0 lvl,
		s.value('@NodeId', 'INT') ParentNodeId,
		s.value('@NodeId', 'INT') NodeId,
		s.value('@PhysicalOp', 'nvarchar(256)') PhysicalOp,
		s.value('@LogicalOp', 'nvarchar(256)') LogicalOp,
		s.value('@EstimateRows', 'FLOAT') EstimateRows,
		s.value('@EstimatedRowsRead', 'FLOAT') EstimatedRowsRead,
		s.value('@EstimateIO', 'FLOAT') EstimateIO,
		s.value('@EstimateCPU', 'FLOAT') EstimateCPU,
		s.value('@AvgRowSize', 'FLOAT') AvgRowSize,
		s.value('@EstimatedTotalSubtreeCost', 'FLOAT') EstimatedTotalSubtreeCost,
		s.value('@TableCardinality', 'FLOAT') TableCardinality,
		s.value('@Parallel', 'INT') Parallel,
		s.value('@EstimateRebinds', 'FLOAT') EstimateRebinds,
		s.value('@EstimateRewinds', 'FLOAT') EstimateRewinds,
		s.value('@EstimatedExecutionMode', 'NVARCHAR(128)') EstimatedExecutionMode,
		o.value('@Database', 'NVARCHAR(128)') db,
		o.value('@Schema', 'NVARCHAR(128)') sch,
		o.value('@Table', 'NVARCHAR(128)') tab,
		o.value('@Index', 'NVARCHAR(128)') idx,
		o.value('@Alias', 'NVARCHAR(128)') alias,
		o.value('@IndexKind', 'NVARCHAR(128)') IndexKind,
		o.value('@Storage', 'NVARCHAR(128)') Storage,
		s.query('child::*/xs:RelOp') xsRelOp
	FROM	query_store_query_plan c
		CROSS APPLY c.query_plan.nodes('/xs:ShowPlanXML/xs:BatchSequence/xs:Batch/xs:Statements/*') n(x)
		OUTER APPLY x.nodes('xs:QueryPlan/xs:RelOp') q(s)
		OUTER APPLY s.nodes('child::*/xs:Object') s(o)
	UNION ALL
	SELECT
		r.database_id,
		r.plan_id,
		r.StatementId,
		r.StatementCompId,
		r.QueryHash,
		r.QueryPlanHash,
		r.lvl + 1 lvl,
		r.NodeId,
		s.value('@NodeId', 'INT') NodeId,
		s.value('@PhysicalOp', 'nvarchar(256)') PhysicalOp,
		s.value('@LogicalOp', 'nvarchar(256)') LogicalOp,
		s.value('@EstimateRows', 'FLOAT') EstimateRows,
		s.value('@EstimatedRowsRead', 'FLOAT') EstimatedRowsRead,
		s.value('@EstimateIO', 'FLOAT') EstimateIO,
		s.value('@EstimateCPU', 'FLOAT') EstimateCPU,
		s.value('@AvgRowSize', 'FLOAT') AvgRowSize,
		s.value('@EstimatedTotalSubtreeCost', 'FLOAT') EstimatedTotalSubtreeCost,
		s.value('@TableCardinality', 'FLOAT') TableCardinality,
		s.value('@Parallel', 'INT') Parallel,
		s.value('@EstimateRebinds', 'FLOAT') EstimateRebinds,
		s.value('@EstimateRewinds', 'FLOAT') EstimateRewinds,
		s.value('@EstimatedExecutionMode', 'NVARCHAR(128)') EstimatedExecutionMode,
		o.value('@Database', 'NVARCHAR(128)') db,
		o.value('@Schema', 'NVARCHAR(128)') sch,
		o.value('@Table', 'NVARCHAR(128)') tab,
		o.value('@Index', 'NVARCHAR(128)') idx,
		o.value('@Alias', 'NVARCHAR(128)') alias,
		o.value('@IndexKind', 'NVARCHAR(128)') IndexKind,
		o.value('@Storage', 'NVARCHAR(128)') Storage,
		s.query('child::*/xs:RelOp') xsRelOp
	FROM	relop r
		CROSS APPLY r.xsRelOp.nodes('xs:RelOp') q(s)
		OUTER APPLY s.nodes('child::*/xs:Object') s(o)
)
INSERT	query_plan_relop(database_id, plan_id, StatementId, StatementCompId, QueryHash, QueryPlanHash, lvl, ParentNodeId, NodeId, PhysicalOp, LogicalOp, EstimateRows, EstimatedRowsRead, EstimateIO, EstimateCPU, AvgRowSize, EstimatedTotalSubtreeCost, TableCardinality, Parallel, EstimateRebinds, EstimateRewinds, EstimatedExecutionMode, db, sch, tab, idx, alias, IndexKind, Storage)
SELECT DISTINCT
	r.database_id,
	r.plan_id,
	r.StatementId,
	r.StatementCompId,
	r.QueryHash,
	r.QueryPlanHash,
	r.lvl,
	r.ParentNodeId,
	r.NodeId,
	r.PhysicalOp,
	r.LogicalOp,
	r.EstimateRows,
	r.EstimatedRowsRead,
	r.EstimateIO,
	r.EstimateCPU,
	r.AvgRowSize,
	r.EstimatedTotalSubtreeCost,
	r.TableCardinality,
	r.Parallel,
	r.EstimateRebinds,
	r.EstimateRewinds,
	r.EstimatedExecutionMode,
	r.db,
	r.sch,
	r.tab,
	r.idx,
	r.alias,
	r.IndexKind,
	r.Storage
FROM	relop r
OPTION(MAXRECURSION 0);


### Cached plans

#### query_plan_statement

In [3]:
WITH XMLNAMESPACES('http://schemas.microsoft.com/sqlserver/2004/07/showplan' AS xs)
INSERT	query_plan_statement(database_id, plan_id, StatementId, StatementCompId, StatementType, StatementSqlHandle, DatabaseContextSettingsId, ParentObjectId, StatementParameterizationType, RetrievedFromCache, StatementSubTreeCost, StatementEstRows, StatementOptmLevel, QueryHash, QueryPlanHash, CardinalityEstimationModelVersion, StatementText, CachedPlanSize, CompileTime, CompileCPU, CompileMemory, SerialRequiredMemory, SerialDesiredMemory, GrantedMemory, MaxUsedMemory, EstimatedAvailableMemoryGrant, EstimatedPagesCached, EstimatedAvailableDegreeOfParallelism, MaxCompileMemory)
SELECT
	c.dbid,
	NULL,
	s.value('@StatementId', 'INT') StatementId,
	s.value('@StatementCompId', 'INT') StatementCompId,
	s.value('@StatementType', 'nvarchar(128)') StatementType,
	s.value('@StatementSqlHandle', 'varchar(128)') StatementSqlHandle,
	s.value('@DatabaseContextSettingsId', 'INT') DatabaseContextSettingsId,
	s.value('@ParentObjectId', 'INT') ParentObjectId,
	s.value('@StatementParameterizationType', 'INT') StatementParameterizationType,
	s.value('@RetrievedFromCache', 'BIT') RetrievedFromCache,
	s.value('@StatementSubTreeCost', 'float') StatementSubTreeCost,
	s.value('@StatementEstRows', 'float') StatementEstRows,
	s.value('@StatementOptmLevel', 'varchar(16)') StatementOptmLevel,
	s.value('@QueryHash', 'varchar(16)') QueryHash,
	s.value('@QueryPlanHash', 'varchar(16)') QueryPlanHash,
	s.value('@CardinalityEstimationModelVersion', 'INT') CardinalityEstimationModelVersion,
	s.value('@StatementText', 'nvarchar(max)') StatementText,
	p.value('@CachedPlanSize', 'BIGINT') CachedPlanSize,
	p.value('@CompileTime', 'BIGINT') CompileTime,
	p.value('@CompileCPU', 'BIGINT') CompileCPU,
	p.value('@CompileMemory', 'BIGINT') CompileMemory,
	p.value('xs:MemoryGrantInfo[1]/@SerialRequiredMemory', 'BIGINT') SerialRequiredMemory,
	p.value('xs:MemoryGrantInfo[1]/@SerialDesiredMemory', 'BIGINT') SerialDesiredMemory,
	p.value('xs:MemoryGrantInfo[1]/@GrantedMemory', 'BIGINT') GrantedMemory,
	p.value('xs:MemoryGrantInfo[1]/@MaxUsedMemory', 'BIGINT') MaxUsedMemory,
	p.value('xs:OptimizerHardwareDependentProperties[1]/@EstimatedAvailableMemoryGrant', 'BIGINT') EstimatedAvailableMemoryGrant,
	p.value('xs:OptimizerHardwareDependentProperties[1]/@EstimatedPagesCached', 'BIGINT') EstimatedPagesCached,
	p.value('xs:OptimizerHardwareDependentProperties[1]/@EstimatedAvailableDegreeOfParallelism', 'BIGINT') EstimatedAvailableDegreeOfParallelism,
	p.value('xs:OptimizerHardwareDependentProperties[1]/@MaxCompileMemory', 'BIGINT') MaxCompileMemory
FROM	cached_plans c
	CROSS APPLY c.query_plan.nodes('/xs:ShowPlanXML/xs:BatchSequence/xs:Batch/xs:Statements/*') n(s)
	OUTER APPLY s.nodes('xs:QueryPlan') q(p);

#### query_plan_relop

In [ ]:
WITH XMLNAMESPACES('http://schemas.microsoft.com/sqlserver/2004/07/showplan' AS xs)
, relop AS (
	SELECT
		c.dbid database_id,
		NULL plan_id,
		x.value('@StatementId', 'INT') StatementId,
		x.value('@StatementCompId', 'INT') StatementCompId,
		x.value('@QueryHash', 'varchar(16)') QueryHash,
		x.value('@QueryPlanHash', 'varchar(16)') QueryPlanHash,
		0 lvl,
		s.value('@NodeId', 'INT') ParentNodeId,
		s.value('@NodeId', 'INT') NodeId,
		s.value('@PhysicalOp', 'nvarchar(256)') PhysicalOp,
		s.value('@LogicalOp', 'nvarchar(256)') LogicalOp,
		s.value('@EstimateRows', 'FLOAT') EstimateRows,
		s.value('@EstimatedRowsRead', 'FLOAT') EstimatedRowsRead,
		s.value('@EstimateIO', 'FLOAT') EstimateIO,
		s.value('@EstimateCPU', 'FLOAT') EstimateCPU,
		s.value('@AvgRowSize', 'FLOAT') AvgRowSize,
		s.value('@EstimatedTotalSubtreeCost', 'FLOAT') EstimatedTotalSubtreeCost,
		s.value('@TableCardinality', 'FLOAT') TableCardinality,
		s.value('@Parallel', 'INT') Parallel,
		s.value('@EstimateRebinds', 'FLOAT') EstimateRebinds,
		s.value('@EstimateRewinds', 'FLOAT') EstimateRewinds,
		s.value('@EstimatedExecutionMode', 'NVARCHAR(128)') EstimatedExecutionMode,
		o.value('@Database', 'NVARCHAR(128)') db,
		o.value('@Schema', 'NVARCHAR(128)') sch,
		o.value('@Table', 'NVARCHAR(128)') tab,
		o.value('@Index', 'NVARCHAR(128)') idx,
		o.value('@Alias', 'NVARCHAR(128)') alias,
		o.value('@IndexKind', 'NVARCHAR(128)') IndexKind,
		o.value('@Storage', 'NVARCHAR(128)') Storage,
		s.query('child::*/xs:RelOp') xsRelOp
	FROM	cached_plans c
		CROSS APPLY c.query_plan.nodes('/xs:ShowPlanXML/xs:BatchSequence/xs:Batch/xs:Statements/*') n(x)
		OUTER APPLY x.nodes('xs:QueryPlan/xs:RelOp') q(s)
		OUTER APPLY s.nodes('child::*/xs:Object') s(o)
	UNION ALL
	SELECT
		r.database_id,
		r.plan_id,
		r.StatementId,
		r.StatementCompId,
		r.QueryHash,
		r.QueryPlanHash,
		r.lvl + 1 lvl,
		r.NodeId,
		s.value('@NodeId', 'INT') NodeId,
		s.value('@PhysicalOp', 'nvarchar(256)') PhysicalOp,
		s.value('@LogicalOp', 'nvarchar(256)') LogicalOp,
		s.value('@EstimateRows', 'FLOAT') EstimateRows,
		s.value('@EstimatedRowsRead', 'FLOAT') EstimatedRowsRead,
		s.value('@EstimateIO', 'FLOAT') EstimateIO,
		s.value('@EstimateCPU', 'FLOAT') EstimateCPU,
		s.value('@AvgRowSize', 'FLOAT') AvgRowSize,
		s.value('@EstimatedTotalSubtreeCost', 'FLOAT') EstimatedTotalSubtreeCost,
		s.value('@TableCardinality', 'FLOAT') TableCardinality,
		s.value('@Parallel', 'INT') Parallel,
		s.value('@EstimateRebinds', 'FLOAT') EstimateRebinds,
		s.value('@EstimateRewinds', 'FLOAT') EstimateRewinds,
		s.value('@EstimatedExecutionMode', 'NVARCHAR(128)') EstimatedExecutionMode,
		o.value('@Database', 'NVARCHAR(128)') db,
		o.value('@Schema', 'NVARCHAR(128)') sch,
		o.value('@Table', 'NVARCHAR(128)') tab,
		o.value('@Index', 'NVARCHAR(128)') idx,
		o.value('@Alias', 'NVARCHAR(128)') alias,
		o.value('@IndexKind', 'NVARCHAR(128)') IndexKind,
		o.value('@Storage', 'NVARCHAR(128)') Storage,
		s.query('child::*/xs:RelOp') xsRelOp
	FROM	relop r
		CROSS APPLY r.xsRelOp.nodes('xs:RelOp') q(s)
		OUTER APPLY s.nodes('child::*/xs:Object') s(o)
)
INSERT	query_plan_relop(database_id, plan_id, StatementId, StatementCompId, QueryHash, QueryPlanHash, lvl, ParentNodeId, NodeId, PhysicalOp, LogicalOp, EstimateRows, EstimatedRowsRead, EstimateIO, EstimateCPU, AvgRowSize, EstimatedTotalSubtreeCost, TableCardinality, Parallel, EstimateRebinds, EstimateRewinds, EstimatedExecutionMode, db, sch, tab, idx, alias, IndexKind, Storage)
SELECT DISTINCT
	r.database_id,
	r.plan_id,
	r.StatementId,
	r.StatementCompId,
	r.QueryHash,
	r.QueryPlanHash,
	r.lvl,
	r.ParentNodeId,
	r.NodeId,
	r.PhysicalOp,
	r.LogicalOp,
	r.EstimateRows,
	r.EstimatedRowsRead,
	r.EstimateIO,
	r.EstimateCPU,
	r.AvgRowSize,
	r.EstimatedTotalSubtreeCost,
	r.TableCardinality,
	r.Parallel,
	r.EstimateRebinds,
	r.EstimateRewinds,
	r.EstimatedExecutionMode,
	r.db,
	r.sch,
	r.tab,
	r.idx,
	r.alias,
	r.IndexKind,
	r.Storage
FROM	relop r
OPTION(MAXRECURSION 0);
